In [1]:
import json, itertools

In [948]:
from pprint import pprint

In [425]:
!python -m pip install coreferee

/usr/bin/python: No module named pip


In [915]:
import spacy, torch

nlp = spacy.load("en_core_web_lg")
activated = spacy.prefer_gpu()

ModuleNotFoundError: No module named 'neuralcoref'

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda:0


### Data Loading/Preprocessing

In [929]:
!ls ../data/ADE20K-pairs/captions/


/project/dataset-exploration/clip-finetune-ade20k/data/coref


In [930]:

train_captions = '../data/ADE20K-pairs/captions/ade20k_train_captions.jsonl'
val_captions = '../data/ADE20K-pairs/captions/ade20k_validation_captions.jsonl'

In [933]:
def read_captions(fname):
    img_paths = []
    captions = []
    with open(fname, 'r') as json_file:
        json_list = list(json_file)
    for json_str in json_list:
        result = json.loads(json_str)
        img_path = result['image_id']
        caption = result['caption'] 
        img_paths.append(img_path)
        captions.append(caption)
    return img_paths, captions

In [6]:
imgs, captions = read_captions(train_captions)

In [941]:
from spacy.lang.en import English
basic_nlp = English()  # just the language with no pipeline
basic_nlp.add_pipe("sentencizer")

def simple_split(doc):
    doc = basic_nlp(doc)
    return [sent.text for sent in doc.sents]

In [942]:
simple_split(captions[0])

['In this picture I can see the inside view of a room, there is a bed with pillows and blankets on it, there are lamps, there is a photo frame attached to the wall, there is a curtain and a window.',
 'Through the window I can see water and the sky.']

In [7]:
imgs[0], captions[0]

('ADE_train_00003661',
 'In this picture I can see the inside view of a room, there is a bed with pillows and blankets on it, there are lamps, there is a photo frame attached to the wall, there is a curtain and a window. Through the window I can see water and the sky.')

In [435]:
import pandas as pd

train_df = pd.DataFrame([imgs,captions]).T

train_df = train_df.sort_values(0)

train_df.columns = ['img', 'caption']

train_df.shape

(20476, 2)

In [450]:
(train_df[train_df.duplicated(subset = ['img'])][:5])

,img,caption
5459,ADE_train_00000031,In the center of the image we can see shower a...
9045,ADE_train_00000031,In this picture I can see there is a bath area...
19572,ADE_train_00000031,"In this image, we can see a shower room and th..."
12646,ADE_train_00000076,In this image we can see a cabinet with wash b...
160,ADE_train_00000076,"In the image I can see cupboards, sink, tap an..."


In [972]:
list(train_df[train_df.img == "ADE_train_00006840"].caption)

['In this picture I can see some people are sitting in front of the tables, among one person is standing and talking in front of mike.']

In [963]:
# !ls ../ADE20K-pairs/preprocessed/training

In [10]:
val_imgs, val_captions = read_captions(val_captions)

val_df = pd.DataFrame([val_imgs,val_captions]).T

val_df = val_df.sort_values(0)

val_df.columns = ['img', 'caption']

#drop duplicates

In [11]:
val_df = val_df.drop_duplicates(subset=['img'])
val_df.shape

(2000, 2)

In [12]:
val_df.head()

,img,caption
1453,ADE_val_00000001,"In this image there is a building, in front of..."
1874,ADE_val_00000002,"At the bottom of the picture, we see the trees..."
1864,ADE_val_00000003,"In this image, we can see vehicles on the road..."
904,ADE_val_00000004,"At the bottom of the picture, we see the grass..."
1830,ADE_val_00000005,This is an inside of an airplane. In this pict...


### Coreference Resolution

In [499]:
# !pip install allennlp allennlp-models

In [500]:
# !pip install --upgrade google-api-python-client

In [459]:
# !pip install "google-cloud-datastore>=1.15.3,<2.0"

In [913]:
# from allennlp.predictors.predictor import Predictor

In [463]:
# !python3 -m allennlp test-install

In [454]:
coref_path = "https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2021.03.10.tar.gz"

In [495]:
coref_path = "/project/dataset-exploration/clip-finetune-ade20k/data/coref"
coref_path = coref_path+"coref-spanbert-large-2021.03.10.tar.gz"

In [496]:
coref = Predictor.from_path(coref_path)  # load the model

NameError: name 'Predictor' is not defined

In [486]:
%cd ../data/

/project/dataset-exploration/clip-finetune-ade20k/data


In [489]:
%cd coref

/project/dataset-exploration/clip-finetune-ade20k/data/coref


In [494]:
# !wget https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2021.03.10.tar.gz

!ls

coref-spanbert-large-2021.03.10.tar.gz


### Split captions on .

In [839]:
def split_captions(doc):
    splits = doc.split(".")
    splits = [s.strip() for s in splits]
    splits = [s for s in splits if len(s)>0]
    return splits
        
train_splits_simple = [split_captions(caption) for caption in train_df.caption]

In [840]:
len([s for s in train_splits_simple if len(s) > 2])

10726

In [841]:
xx = [s for s in train_splits_simple if len(s) > 2]

In [883]:
merged = list(itertools.chain.from_iterable(xx))
j = [x for x in merged if len(x.split()) > 10]
k = [x for x in j if "," in x]

In [884]:
from collections import Counter

In [885]:
minus_1 = []
minus_2 = []
plus_1 = []
plus_2 = []
minus_pairs = []
plus_pairs = []
window1_pairs = []
window2_pairs = []
for line_count, x in enumerate(k):
    nlp_doc = nlp(x)
    tags = [i.tag_ for i in nlp_doc]
    text = [i.text for i in nlp_doc]
    if line_count < 3000:
        print()
        print(nlp_doc)
#     print(' '.join(tags))
    try:
        for i, tok in enumerate(nlp_doc):
            if tok.text == ",":
                minus_1.append(tags[i-1])
                minus_2.append(tags[i-2])
                plus_1.append(tags[i+1])
                plus_2.append(tags[i+2])
                minus_pairs.append((tags[i-1],tags[i-2]))
                plus_pairs.append((tags[i+1],tags[i+2]))
                window1_pairs.append((tags[i-1],tags[i+1]))
                window2_pairs.append((tags[i-2],tags[i-1],tags[i+1],tags[i+2]))
                if line_count < 3000:
                    print(window2_pairs[-1])
                    print(dict(zip(nlp_doc[i-2:i+3], tags[i-2:i+3])))
    except IndexError as e:
        pass
#     print(' '.join(text))


On the left side of the image we can see plants in pots, some lights
('IN', 'NNS', 'DT', 'NNS')
{in: 'IN', pots: 'NNS', ,: ',', some: 'DT', lights: 'NNS'}

In the middle, there is a door and a wall clock
('DT', 'NN', 'EX', 'VBZ')
{the: 'DT', middle: 'NN', ,: ',', there: 'EX', is: 'VBZ'}

At the top, there is a roof along with the lamps
('DT', 'NN', 'EX', 'VBZ')
{the: 'DT', top: 'NN', ,: ',', there: 'EX', is: 'VBZ'}

At the bottom, I can see a commode and a pot on the floor
('DT', 'NN', 'PRP', 'MD')
{the: 'DT', bottom: 'NN', ,: ',', I: 'PRP', can: 'MD'}

In the middle, I can see a wash basin with taps, a flower vase and some objects
('DT', 'NN', 'PRP', 'MD')
{the: 'DT', middle: 'NN', ,: ',', I: 'PRP', can: 'MD'}
('IN', 'NNS', 'DT', 'NN')
{with: 'IN', taps: 'NNS', ,: ',', a: 'DT', flower: 'NN'}

At the top, I can see a lamp on a roof
('DT', 'NN', 'PRP', 'MD')
{the: 'DT', top: 'NN', ,: ',', I: 'PRP', can: 'MD'}

In the mirror I can see the reflection of clothes, door and the frame to the


In this image, we can see a bathtub and some bottles
('DT', 'NN', 'PRP', 'MD')
{this: 'DT', image: 'NN', ,: ',', we: 'PRP', can: 'MD'}

Also we can see photo frame, clock and some other objects in the cupboard
('NN', 'NN', 'NN', 'CC')
{photo: 'NN', frame: 'NN', ,: ',', clock: 'NN', and: 'CC'}

In this picture we can see the taps, sinks, platforms, a toilet seat, baskets, a bathtub with a tap, a box and a cloth
('DT', 'NNS', 'NNS', ',')
{the: 'DT', taps: 'NNS', ,: ',', sinks: 'NNS', ,: ','}
(',', 'NNS', 'NNS', ',')
{,: ',', sinks: 'NNS', ,: ',', platforms: 'NNS', ,: ','}
(',', 'NNS', 'DT', 'NN')
{,: ',', platforms: 'NNS', ,: ',', a: 'DT', toilet: 'NN'}
('NN', 'NN', 'NNS', ',')
{toilet: 'NN', seat: 'NN', ,: ',', baskets: 'NNS', ,: ','}
(',', 'NNS', 'DT', 'NN')
{,: ',', baskets: 'NNS', ,: ',', a: 'DT', bathtub: 'NN'}
('DT', 'NN', 'DT', 'NN')
{a: 'DT', tap: 'NN', ,: ',', a: 'DT', box: 'NN'}

We can see the flower vases, a soap with a soap box, boxes and some objects on the platforms
('NN'


Through the windows, I can see 2 chairs and a table, number of plants, trees, walls and the sky
('DT', 'NNS', 'PRP', 'MD')
{the: 'DT', windows: 'NNS', ,: ',', I: 'PRP', can: 'MD'}
('DT', 'NN', 'NN', 'IN')
{a: 'DT', table: 'NN', ,: ',', number: 'NN', of: 'IN'}
('IN', 'NNS', 'NNS', ',')
{of: 'IN', plants: 'NNS', ,: ',', trees: 'NNS', ,: ','}
(',', 'NNS', 'NNS', 'CC')
{,: ',', trees: 'NNS', ,: ',', walls: 'NNS', and: 'CC'}

On the left side of this picture, I can see a socket and a light on the wall
('DT', 'NN', 'PRP', 'MD')
{this: 'DT', picture: 'NN', ,: ',', I: 'PRP', can: 'MD'}

In the front, we can see a window along with the toilet paper
('DT', 'NN', 'PRP', 'MD')
{the: 'DT', front: 'NN', ,: ',', we: 'PRP', can: 'MD'}

In the bottom right corner of the image there is a bathtub,cloth and bottle
('DT', 'NN', 'NN', 'CC')
{a: 'DT', bathtub: 'NN', ,: ',', cloth: 'NN', and: 'CC'}

In this picture we can see the taps, a wash basin, a flush tank, a bottle and a few things in the bottom right


On the wall we can see tissue holder, shower, tap, towel on a rod and some other object
('NN', 'NN', 'NN', ',')
{tissue: 'NN', holder: 'NN', ,: ',', shower: 'NN', ,: ','}
(',', 'NN', 'NN', ',')
{,: ',', shower: 'NN', ,: ',', tap: 'NN', ,: ','}
(',', 'NN', 'NN', 'IN')
{,: ',', tap: 'NN', ,: ',', towel: 'NN', on: 'IN'}

In the middle of the picture, we see a wash basin and a tap
('DT', 'NN', 'PRP', 'VBP')
{the: 'DT', picture: 'NN', ,: ',', we: 'PRP', see: 'VBP'}

Behind that, we see a wall which is made up of tiles
('IN', 'DT', 'PRP', 'VBP')
{Behind: 'IN', that: 'DT', ,: ',', we: 'PRP', see: 'VBP'}

On the right side, we see the glass wall and a bathtub
('JJ', 'NN', 'PRP', 'VBP')
{right: 'JJ', side: 'NN', ,: ',', we: 'PRP', see: 'VBP'}

In this picture we can see the wall, glass door, window, cupboards, toilet seat, sink, door
('DT', 'NN', 'NN', 'NN')
{the: 'DT', wall: 'NN', ,: ',', glass: 'NN', door: 'NN'}
('NN', 'NN', 'NN', ',')
{glass: 'NN', door: 'NN', ,: ',', window: 'NN', ,: ','}



Right side there is a table having flower vases, lamps and mirror on it
('NN', 'NNS', 'NNS', 'CC')
{flower: 'NN', vases: 'NNS', ,: ',', lamps: 'NNS', and: 'CC'}

In the foreground, I can see a couch and some wooden objects having some paintings
('DT', 'NN', 'PRP', 'MD')
{the: 'DT', foreground: 'NN', ,: ',', I: 'PRP', can: 'MD'}

In the background, I can see tables with a TV, houseplants, photo frames and other things
('DT', 'NN', 'PRP', 'MD')
{the: 'DT', background: 'NN', ,: ',', I: 'PRP', can: 'MD'}
('DT', 'NN', 'NNS', ',')
{a: 'DT', TV: 'NN', ,: ',', houseplants: 'NNS', ,: ','}
(',', 'NNS', 'NN', 'NNS')
{,: ',', houseplants: 'NNS', ,: ',', photo: 'NN', frames: 'NNS'}

We can also see the windows, lights and a ceiling fan to a roof
('DT', 'NNS', 'NNS', 'CC')
{the: 'DT', windows: 'NNS', ,: ',', lights: 'NNS', and: 'CC'}

On the right side we can see some plants, a group of trees, grass, a wooden fence and the sky
('DT', 'NNS', 'DT', 'NN')
{some: 'DT', plants: 'NNS', ,: ',', a: 'DT', g

...


In the middle of this image, there is a white wall which is having two windows
('DT', 'NN', 'EX', 'VBZ')
{this: 'DT', image: 'NN', ,: ',', there: 'EX', is: 'VBZ'}

This is a picture taken inside of a room, in this image on the left side there is a bed
('DT', 'NN', 'IN', 'DT')
{a: 'DT', room: 'NN', ,: ',', in: 'IN', this: 'DT'}

On the bed there are blankets, pillows and also there is a photo frame and objects
('VBP', 'NNS', 'NNS', 'CC')
{are: 'VBP', blankets: 'NNS', ,: ',', pillows: 'NNS', and: 'CC'}

And in the background there is wall, and on the right side of the image there are photo frames, candles and there are objects, wooden cupboard, basket, photo frames
('VBZ', 'NN', 'CC', 'IN')
{is: 'VBZ', wall: 'NN', ,: ',', and: 'CC', on: 'IN'}
('NN', 'NNS', 'NNS', 'CC')
{photo: 'NN', frames: 'NNS', ,: ',', candles: 'NNS', and: 'CC'}
('VBP', 'NNS', 'JJ', 'NN')
{are: 'VBP', objects: 'NNS', ,: ',', wooden: 'JJ', cupboard: 'NN'}
('JJ', 'NN', 'NN', ',')
{wooden: 'JJ', cupboard: 'NN', ,: ',', 


In this image we can see a bed with some pillars, a table and a cushion on a chair which are placed on the floor
('DT', 'NNS', 'DT', 'NN')
{some: 'DT', pillars: 'NNS', ,: ',', a: 'DT', table: 'NN'}

We can also see the lamps and the photo frames on the tables, a bowl and a photo frame on a cupboard, a mirror and a photo frame on a wall
('DT', 'NNS', 'DT', 'NN')
{the: 'DT', tables: 'NNS', ,: ',', a: 'DT', bowl: 'NN'}
('DT', 'NN', 'DT', 'NN')
{a: 'DT', cupboard: 'NN', ,: ',', a: 'DT', mirror: 'NN'}

We can also see a door, windows and a roof with some ceiling lights and a chandelier
('DT', 'NN', 'NNS', 'CC')
{a: 'DT', door: 'NN', ,: ',', windows: 'NNS', and: 'CC'}

At the top of the image we can see the roof, lights and a ceiling fan
('DT', 'NN', 'NNS', 'CC')
{the: 'DT', roof: 'NN', ,: ',', lights: 'NNS', and: 'CC'}

In the middle of the image we can see the bed with pillows, cushions and a bed sheet on it
('IN', 'NNS', 'NNS', 'CC')
{with: 'IN', pillows: 'NNS', ,: ',', cushions: 'NNS', 


In the middle of the picture there are windows, air-conditioner, wall and various objects
('VBP', 'NNS', 'NN', 'HYPH')
{are: 'VBP', windows: 'NNS', ,: ',', air: 'NN', -: 'HYPH'}
('HYPH', 'NN', 'NN', 'CC')
{-: 'HYPH', conditioner: 'NN', ,: ',', wall: 'NN', and: 'CC'}

On the right side of the image we can see lamp and books placed on the table, chair and dustbin
('DT', 'NN', 'NN', 'CC')
{the: 'DT', table: 'NN', ,: ',', chair: 'NN', and: 'CC'}

In this image there is a bed on the bed there is blanket, and some pillows
('VBZ', 'NN', 'CC', 'DT')
{is: 'VBZ', blanket: 'NN', ,: ',', and: 'CC', some: 'DT'}

And beside the bed there are cupboards, lamp, and some objects wall
('VBP', 'NNS', 'NN', ',')
{are: 'VBP', cupboards: 'NNS', ,: ',', lamp: 'NN', ,: ','}
(',', 'NN', 'CC', 'DT')
{,: ',', lamp: 'NN', ,: ',', and: 'CC', some: 'DT'}

In this image, we can see a bed and there is a light and some objects on the stand
('DT', 'NN', 'PRP', 'MD')
{this: 'DT', image: 'NN', ,: ',', we: 'PRP', can: 'MD


In this image, I can see two beds with the bed sheets and pillows on it
('DT', 'NN', 'PRP', 'MD')
{this: 'DT', image: 'NN', ,: ',', I: 'PRP', can: 'MD'}

On the left side of the image, I can see two lamps and there are few objects on a table
('DT', 'NN', 'PRP', 'MD')
{the: 'DT', image: 'NN', ,: ',', I: 'PRP', can: 'MD'}

Beside the bed, I can see a lamp, book and few other things on a table and there is a chair
('DT', 'NN', 'PRP', 'MD')
{the: 'DT', bed: 'NN', ,: ',', I: 'PRP', can: 'MD'}
('DT', 'NN', 'NN', 'CC')
{a: 'DT', lamp: 'NN', ,: ',', book: 'NN', and: 'CC'}

On the right side of the image, I can see a wooden object and a couch on the floor
('DT', 'NN', 'PRP', 'MD')
{the: 'DT', image: 'NN', ,: ',', I: 'PRP', can: 'MD'}

In this picture I can see a bed, few pillows and blankets on the bed
('DT', 'NN', 'JJ', 'NNS')
{a: 'DT', bed: 'NN', ,: ',', few: 'JJ', pillows: 'NNS'}

I can see a table with drawers on the left side, it looks like a flower pot on the table
('JJ', 'NN', 'PRP', 'V


On the left side, we see a wall and a table
('JJ', 'NN', 'PRP', 'VBP')
{left: 'JJ', side: 'NN', ,: ',', we: 'PRP', see: 'VBP'}

In the image there is a bed with pillows and a tray with glass, a bottle and other objects
('IN', 'NN', 'DT', 'NN')
{with: 'IN', glass: 'NN', ,: ',', a: 'DT', bottle: 'NN'}

On the sides of the bed there are small cupboards with lamps, a statue and a vase with flowers
('IN', 'NNS', 'DT', 'NN')
{with: 'IN', lamps: 'NNS', ,: ',', a: 'DT', statue: 'NN'}

At the bottom of the picture, we see a stool on which a tray containing the objects is placed
('DT', 'NN', 'PRP', 'VBP')
{the: 'DT', picture: 'NN', ,: ',', we: 'PRP', see: 'VBP'}

On the left side, we see drawers and the pillows in black and yellow color
('JJ', 'NN', 'PRP', 'VBP')
{left: 'JJ', side: 'NN', ,: ',', we: 'PRP', see: 'VBP'}

In this image there are pillows, book and some objects on the beds
('VBP', 'NNS', 'NN', 'CC')
{are: 'VBP', pillows: 'NNS', ,: ',', book: 'NN', and: 'CC'}

There are lamps, laptop

In [901]:
minus_1 = Counter(minus_1)
minus_2 = Counter(minus_2)
plus_1 = Counter(plus_1)
plus_2 = Counter(plus_2)
minus_pairs = Counter(minus_pairs)
plus_pairs = Counter(plus_pairs)
window1_pairs = Counter(window1_pairs)
window2_pairs = Counter(window2_pairs)
print("minus 2: ", minus_2)
print("minus 1: ", minus_1)
print("plus 1: ", plus_1)
print("plus 2: ", plus_2)
print()
print("minus_pairs: ", minus_pairs)
print()
print("plus_pairs: ", plus_pairs)
print()
print("window1_pairs: ", window1_pairs)
print()
print("window2_pairs: ", window2_pairs)

minus 2:  Counter({'DT': 8718, ',': 5329, 'NN': 1664, 'JJ': 1418, 'VB': 1353, 'IN': 1205, 'VBP': 953, 'CC': 205, 'VBZ': 181, 'CD': 97, 'VBG': 81, 'NNP': 53, 'NNS': 45, 'WDT': 25, 'VBN': 24, 'HYPH': 17, 'JJR': 13, 'PRP$': 13, 'PRP': 7, 'VBD': 6, 'RB': 4, 'MD': 3, 'TO': 3, 'POS': 1})
minus 1:  Counter({'NN': 12157, 'NNS': 8414, 'DT': 221, 'PRP': 158, 'JJ': 109, 'VBG': 94, 'VB': 50, 'WDT': 47, 'NNP': 38, 'UH': 33, 'VBZ': 32, 'VBN': 30, 'RB': 13, 'VBP': 6, 'POS': 5, 'IN': 3, 'MD': 3, 'RP': 2, 'WP': 1, 'JJR': 1, ',': 1})
plus 1:  Counter({'NNS': 5952, 'NN': 4921, 'PRP': 3988, 'DT': 1894, 'EX': 1702, 'IN': 985, 'JJ': 743, 'CC': 596, 'VB': 115, 'VBZ': 102, 'VBG': 100, 'CD': 92, 'WDT': 85, 'NNP': 55, 'VBP': 21, 'WRB': 16, 'RB': 16, 'VBN': 15, 'UH': 4, '_SP': 4, 'WP': 3, 'MD': 2, 'VBD': 2, 'NNPS': 1, ',': 1})
plus 2:  Counter({',': 5329, 'CC': 3653, 'NN': 2380, 'VBP': 2369, 'MD': 2318, 'NNS': 1477, 'VBZ': 1164, 'IN': 982, 'DT': 849, 'JJ': 159, 'PRP': 135, 'EX': 114, 'VBN': 51, 'VBG': 27, 'RB': 

In [920]:
# def rulebook(prev_tag, next_tag, next_next_tag):
#     prev_tag = prev_tag.replace('NNS','NN')
#     next_tag = next_tag.replace('NNS', 'NN')
    
#     duals = [('NN','IN'),('PRP','IN'), ('RB','IN')]
             
#     triples = [('NN','CC','IN'), ('NN','CC','PRP'), ('NN','CC','EX'),\
#                 ('PRP','CC','EX'), ('PRP','CC','PRP'), ('PRP','CC','DT')]
    
#     d = (prev_tag, next_tag)
#     t = (prev_tag, next_tag, next_next_tag)
    
#     return d in duals or t in triples


# def rule_based_split(doc):
#     doc = nlp(doc)
#     toks, tags = [], []
#     output_toks = []
    
#     for tok in doc:
#         toks.append(tok.text)
#         tags.append(tok.tag_)
    
#     for i, tok in enumerate(toks):
#         if tok == ',':
#             tag = tags[i]
            
#             if i > 1 and i < len(toks) - 1:
#                 prev_tag = tags[i-1]
#                 next_tag = tags[i+1]
#                 next_next_tag = tags[i+2]
#             else:
#                 output_toks.append(tok)
#                 continue
            
#             if rulebook(prev_tag, next_tag, next_next_tag):
#                 output_toks.append(".")
#             else:
#                 output_toks.append(tok)
#         else:
#             output_toks.append(tok)
#     return ' '.join(output_toks)

# print(t)
# rule_based_split(t)

In [976]:
# t ='In the middle, I can see a cabinet with a wash basin having a tap, and I can \
# see bottles, a measuring jar, vessels, fruits in a stand, \
# and I can see electronic devices, a chimney and sockets on a wall.'

# t = 'In this image we can see there is a kitchen platform with stove, oven, cupboards and some utensils,, there are other objects on top of the platform, beside the platform there is a table with some objects and lamp, beside that there is a sofa, behind the sofa there is a cupboard and few frames, and few objects are hanging on the wall and there are Windows.'

# nlp_doc = nlp(t)
# tags = [i.tag_ for i in nlp_doc]
# text = [i.text for i in nlp_doc] 
# print(nlp_doc)
# print(' '.join(tags))

# for i, tok in enumerate(nlp_doc):
#     if tok.dep_ == "punct":
#         print(dict(zip(nlp_doc[i-2:i+3], tags[i-2:i+3])))

In [368]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
  
tokenizer = AutoTokenizer.from_pretrained("flax-community/t5-v1_1-base-wikisplit")

model = AutoModelForSeq2SeqLM.from_pretrained("flax-community/t5-v1_1-base-wikisplit")

model = model.to(device)

In [981]:
import glob

# fnames = glob.glob()

def read_file(fname):
    with open(fname, 'r') as f:
        text = f.read()
    return text

def read_dir(dirname):
    fnames = glob.glob(dirname+"*")
    names, captions = [], []
    for fname in fnames:
        n = fname.split("/")[-1].split(".")[0]
        caption = read_file(fname)
        
        names.append(n)
        captions.append(caption)
        
    return (names, captions)


n, c = read_dir("../ADE20K-pairs/preprocessed/step2-output/training/")

In [998]:
l = [line.split(".") for line in c]
l = [item for sublist in l for item in sublist]
temp = [i for i in l if len(i.split()) > 20]

In [999]:
pprint(temp[:-1])

[' On table there is screen, mouse, pad, keyboard, book, three pens, two sound '
 'boxes, and on the right side there is a chair',
 'This picture is clicked outside and we can see the group of people holding '
 'some objects and seems to be walking, and we can see the water and the '
 'houses',
 'In this image on the bed there are pillows and blankets, beside the bed '
 'there is a lamp, cup and an object on the table',
 'In this picture I can see a refrigerator, microwave oven, there are chairs, '
 'there is a board attached to the wall',
 'It seems like stools and a table in the foreground area of the image, there '
 'are windows, sofa, lamps and a counter in the background',
 ' in front there is green grass, green leaves for tree, top there is sky with '
 'clouds, and there is flag attached to pole with some text',
 'In this image there are chairs, and in the background there is a wooden '
 'wall, door, lights, boards, and there might be a window',
 ' on the pavement there are sign 

 ' In the background there is a refrigerator and there are platforms, on '
 'platforms there are objects which are white in colour and there is a plate',
 'In this picture, it seems like wooden boards at the bottom side of this '
 'picture, there are vehicles and a roof in the background',
 ' At the bottom of this image, there are flower pots with plants, a dustbin, '
 'stairs, and I can see two persons standing',
 ' above a cabinet there are some books, in the background there is a wall and '
 'there are some frames attached to a wall',
 'In this image we can see two wooden beds with pink color pillows and black '
 'and white check bed sheet, we can also see two pink color clothes, a lamp '
 'light, a flower vase, a wooden drawer, two white color walls, a wooden '
 'carpeted floor, and something is written on the bottom right corner of this '
 'image',
 'In the foreground, I can see a fence, grass, flowering plants, and a person '
 'wearing a bag is holding a camera in hand',
 ' And I

 'On the left side of the image, we can see some people are walking on the '
 'road, and some of some people are wearing jackets and carrying bags',
 "In this image I can see few park 's rides, and I can also see few children "
 "are sitting on few park 's rides",
 ' On the both sides of this image I can see two benches, and on two benches I '
 'can see few people are sitting',
 ' in front of a person there are screens are attached to the back of the two '
 'seats, there are doors and windows of the car',
 ' I can see a board to the pole with some text on a board to the pole with '
 'some text on it',
 ' in which I can see few cannons, which are black in color, the wooden floor, '
 'few stairs, few persons standing on few stairs, few ropes, and in the '
 'background I can see the sky',
 'In this image there is a hall, there are some lights visible at the roof, '
 'there are some rods visible, there are some posters attached to the wall, '
 'there is the text',
 ' We can also see a grou

 'ceiling',
 ' In the background we can see the wall, windows, curtains, roof, ceiling '
 'lights, and some pictures of persons, and pictures of some objects on the '
 'wall',
 'This is an inside view of a party bus, and here we can see lights, boards, '
 'seats, windows, and there is a screen',
 ' and I can see buildings with windows, balconies, metal poles, metal gates '
 'and some objects on walls, and I can see plants',
 ' In this picture we can see chairs and we can see the reflection of a man, '
 'chairs, table, lights, showpieces, houseplant and a few objects in a mirror',
 ' In the background, there are books and we can see toys, lights, a mirror '
 'and some objects, and there is a wall',
 'In this picture I can observe a room in which I can observe an oven in the '
 'middle of this picture',
 'In this picture, we can see the view of bakery, where we can see so many '
 'food items like snacks placed in shelves with price',
 'In the foreground, I can see some text on boards, an

In [1000]:
len(temp)

3104

In [419]:
complex_sentence = "In the middle of the image we can see the bed with mattress and bed sheets"
complex_sentence = 'In the background there is like a stall and on the right side of the image there are like a countertop, above them there are a few boards with some text on it, there is a rack, behind that there is a tree and there are a few rods connected to each other with a cloth, behind that there are two vehicles are parked and there are a few banners with some images and text are hanging on the wall'
# complex_sentence = "We can see another lamp and decorative items on an object visible in the bottom right"

# complex_sentence = "In this image there are sofas with pillows on it, in front of that there is a table with some plant pot on it, beside the sofa there is a lamp and in the background there is a fireplace, beside that there is a tree and there is a depiction of a duck, there is a frame hanging on the wall and on the left side of the image we can see the leaves of a tree"

complex_sentence =  'On the right side of the image there is a table and there is a box and there is a toy and there are posters on the wall and there are numbers and symbols on the posters and there is a picture of a tree and there are pictures of fruits and there are pictures of people on the wall and there are windows and curtains and there is a tree behind the window'
# complex_sentence = "On the right side there is stool and on the stool there is a light lamp"

# complex_sentence = 'In this image there is a sofa with a pillow and there is a couch with a pillow and a cloth, in between then there is like a table with an object on the carpet which is on the floor, beside the couch there is a small table with some objects on it, beside that there are a few objects arranged on the shelves, there are so many frames hanging on the wall, behind the couch there is a lamp and a flower pot, there is a fire place, beside that there is an object, stairs and a gate'
# complex_sentence = 'In the background, there is a woman in a t-shirt, watching a book which is on a table on which there are some objects arranged, in front of her, there are chair and a white color sheet, beside this sheet we can see there are some objects, behind this woman we can see there are boards having posters arranged on the ground, there is a person, there are plants and a thread'

complex_sentence = 'In this image there are a few food items are arranged in the rack, behind it there is a person standing and holding some object in his hand, there are a few objects on the table, beside that there is like a refrigerator and there are a few bottles and other objects are arranged in the rack'

# complex_sentence = "On the right side of the picture we can see a partial part of a cupboard"

In [275]:
# nlp_doc = nlp(complex_sentence)
# tags = [i.tag_ for i in nlp_doc]
# # tags
# len(nlp_doc)

# %%time
# t5_split(complex_sentence)

In [751]:
def t5_split(doc):
    docs = []
    
    tokenized = tokenizer(doc, return_tensors="pt").to(device)
    answer = model.generate(tokenized['input_ids'], 
                            attention_mask = tokenized['attention_mask'], 
                            max_length=256, num_beams=20)
    
    generated = tokenizer.decode(answer[0], skip_special_tokens=True)
#     print(doc)
    return [i.strip() for i in generated.split(".") if len(i) > 0]

def call_condition(x):
    nlp_doc = nlp(x)
    tags = [i.tag_ for i in nlp_doc]
    text = [i.text for i in nlp_doc]
    cond = 'CC' and 'IN' in tags
    return len(x) > 10 and cond

def check_doc(d):
    toks = d.lower().split(" ")
    total_toks = len(toks)
    done=False
    for i, tok in enumerate(toks):
        if tok in ["in","on","and","of","are","which","is"]:
            try:
                cond1 = toks[i-1] == toks[i+1]
                if i < total_toks - 2:
                    cond2 = toks[i-2] + toks[i-1] == toks[i+1] + toks[i+2]
                if i < total_toks - 3:
                    cond3 = toks[i-3] + toks[i-2] + toks[i-1] == \
                    toks[i+1] + toks[i+2] + toks[i+3]
                if i < total_toks - 4:
                    cond4 = toks[i-4] + toks[i-3] + toks[i-2] + toks[i-1] == \
                    toks[i+1] + toks[i+2] + toks[i+3] + toks[i+4]
                if i < total_toks - 5:
                    cond5 = toks[i-5] + toks[i-4] + toks[i-3] + toks[i-2] + toks[i-1] \
                    == toks[i+1] + toks[i+2] + toks[i+3] + toks[i+4] + toks[i+5]
                if i < total_toks - 6:
                    cond6 = toks[i-6] + toks[i-5] + toks[i-4] + toks[i-3] + \
                    toks[i-2] + toks[i-1] == toks[i+1] + toks[i+2] + \
                    toks[i+3] + toks[i+4] + toks[i+5] + toks[i+6]
                if i < total_toks - 7:
                    cond7 = toks[i-7] + toks[i-6] + toks[i-5] + toks[i-4] + toks[i-3] + \
                    toks[i-2] + toks[i-1] == toks[i+1] + toks[i+2] + \
                    toks[i+3] + toks[i+4] + toks[i+5] + toks[i+6] + toks[i+7]
                if i < total_toks - 8:
                    cond8 = toks[i-8] + toks[i-7] + toks[i-6] + toks[i-5] +\
                    toks[i-4] + toks[i-3] + \
                    toks[i-2] + toks[i-1] == toks[i+1] + toks[i+2] + \
                    toks[i+3] + toks[i+4] + toks[i+5] + toks[i+6] + toks[i+7] + toks[i+8]
                if cond1 or cond2 or cond3 or cond4 or cond5 or cond6 or cond7 or cond8:
                    done=True
                    d = " ".join(toks[:i-1])
            except Exception as e:
                pass
#                 d = " ".join(toks[:i])
#                     done=True
#                     print(e, " ", i, d)
        if done:
            break

    d0 = d.split(" and ")
    if len(d0) > 1:
        d1 = d0[0].strip().lower()
        d2 = d0[1].strip().lower()
        if d1.endswith(d2):
            d=d1
    return d

In [977]:
context = "In the background I can see few curtains, a door, few candle stands, a fireplace, two lamps"
check_doc(context)

'In the background I can see few curtains, a door, few candle stands, a fireplace, two lamps'

In [788]:
from queue import Queue
def t5_calls(text):
    callables = CheckableQueue()
    outputs = []
#     callables.put(text)
    i=0
    
    s = nlp(text)
    for sent in s.sents:
        if not call_condition(sent.text):
            outputs.append(sent.text)
        else:
            callables.put(sent.text)
#     sent.text for sent in s.sents if not call_condition(sent.text)
    
    print("output: ", outputs)
    print("callables: ", callables.qsize())
    while callables.qsize() > 0:
        if i > 10:
            break
        print("Queue = ", callables.qsize())
        doc = callables.get()
        print("Doc = ", doc)
        if call_condition(doc):
            splits = t5_split(doc)
            for split in splits:
                if call_condition(split) and split not in callables:
                    callables.put(split)
                else:
                    outputs.append(split)
        else:
            outputs.append(doc)
        i+=1
    print("outputs: ", outputs)
        
    return [check_doc(doc) for doc in set(outputs) if len(doc)>0]

c = "This picture describes about inside view of a hall, in this we can find group of people,few are sitting on the chairs, in the background we can see lights."
# t5_split(t5_split())
c = 'This picture describes about inside view of a hall,. in this we can find group of people,few are sitting on the chairs,, in the background we can see lights.'
c = 'in this we can find group of people,few are sitting on the chairs,, in the background we can see lights'
print(c)
t5_calls(c)

in this we can find group of people,few are sitting on the chairs,, in the background we can see lights
output:  []
callables:  1
Queue =  1
Doc =  in this we can find group of people,few are sitting on the chairs,, in the background we can see lights
Queue =  2
Doc =  In this we can find group of people, few are sitting on the chairs
Queue =  3
Doc =  In the background we can see lights, in the background we can see lights
Queue =  3
Doc =  In this we can find group of people
Queue =  3
Doc =  Few are sitting on the chairs
Queue =  4
Doc =  In the background we can see lights
Queue =  4
Doc =  In this we can find group of people
Queue =  4
Doc =  Few are sitting on the chairs
Queue =  4
Doc =  Many are sitting on the chairs
Queue =  4
Doc =  In the background we can see lights
Queue =  4
Doc =  In this we can find group of people
outputs:  ['In the background we can see lights', 'In this we can find group of people', 'In the background we can see lights', 'In this we can find group of

['In this we can find group of people', 'Many are sitting on the chairs', '']

In [787]:
x = [s.strip() for s in c.split(".") if len(s) > 0]

['This picture describes about inside view of a hall,',
 'in this we can find group of people,few are sitting on the chairs,, in the background we can see lights']

In [768]:
# from nemo.collections.nlp.models import PunctuationCapitalizationModel

# # to get the list of pre-trained models
# PunctuationCapitalizationModel.list_available_models()

# # Download and load the pre-trained BERT-based model
# model = PunctuationCapitalizationModel.from_pretrained("punctuation_en_bert")

# # try the model on a few examples
# model.add_punctuation_capitalization(['how are you', 'great how about you'])

In [765]:
# !pip install nemo_toolkit[all]
# !pip install --upgrade protobuf
!pip3 uninstall protobuf

Found existing installation: protobuf 3.17.3
Uninstalling protobuf-3.17.3:
  Would remove:
    /home/users/ragarwal/.local/lib/python3.6/site-packages/google/protobuf/*
    /home/users/ragarwal/.local/lib/python3.6/site-packages/protobuf-3.17.3-py3.6-nspkg.pth
    /home/users/ragarwal/.local/lib/python3.6/site-packages/protobuf-3.17.3.dist-info/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user


In [720]:

    
def t5_recursive_call(doc):
    print(doc)
    docs = []
    callables = [doc]
    prev_callables=[doc]
    i=0
    
    while True:
        print(i)
        print("[callables]=", callables)
        print("[docs]", docs)
        if len(callables) < 1:
            break
        if i > 10:
            break
        
        continue_splitting = False
        for doc in callables:
            doc = check_doc(doc)
            prev_callables = list(set(callables))
            print("\nDOC: ", doc)
            if doc in callables:
                callables.remove(doc)
            if call_condition(doc):
#                 callables.remove(doc)
                splits = t5_split(doc)
                print()
                for s in splits:
                    s = check_doc(s)
                    print("Split:", s)
                    if s in prev_callables:
#                         callables.remove(s)
#                         if s in callables:
#                             callables.remove(s)
                        docs.append(s)
                        continue
                    if call_condition(s):
                        callables.append(s)
                    else:
                        docs.append(s)
            else:
                docs.append(doc)
                if doc in callables:
                    callables.remove(doc)
        i+=1             

    return [check_doc(doc) for doc in set(docs) if len(doc)>0]
        
    

# c = "In this image there are sofas with pillows on sofas with pillows on it, in front of sofas with pillows on it there is a table with some plant pot on a table with some plant pot on it, beside the sofa there is a lamp and in the background there is a fireplace, beside a fireplace there is a tree and there is a depiction of a duck, there is a frame hanging on the wall and on the left side of this image we can see the leaves of a tree"
# c = 
# c = "This picture describes about inside view of a hall, in This picture we can find group of people,few are sitting on the chairs, in the background we can see lights."
c = "This picture describes about inside view of a hall, in this we can find group of people,few are sitting on the chairs, in the background we can see lights."
# t5_split(t5_split())
t5_recursive_call(c)

This picture describes about inside view of a hall, in this we can find group of people,few are sitting on the chairs, in the background we can see lights.
0
[callables]= ['This picture describes about inside view of a hall, in this we can find group of people,few are sitting on the chairs, in the background we can see lights.']
[docs] []

DOC:  This picture describes about inside view of a hall, in this we can find group of people,few are sitting on the chairs, in the background we can see lights.

Split: This picture describes about inside view of a hall
Split: In this we can find group of people, few are sitting on the chairs, in the background we can see lights

DOC:  In this we can find group of people, few are sitting on the chairs, in the background we can see lights

Split: In this we can find group of people, few are sitting on the chairs
Split: In the background we can see lights, in the background we can see lights

DOC:  In the background we can see lights, in the backgroun

['few',
 'In this we can find group of people',
 'Inside view of a hall',
 'This picture describes about inside view of a hall']

In [695]:
def t5_recursive_call(doc):
    print(doc)
    
    docs = []
    callables = [doc]
    i=0
    
    while True:
        print(i)
        print("[callables]=", callables)
        print("[docs]", docs)
        if len(callables) < 1:
            break
        if i > 10:
            break
        
        continue_splitting = False
        for doc in callables:
            callables.remove(doc)
            print()
            splits = t5_split(doc)
            for s in splits:
                s = check_doc(s)
#                     print("s:", s)
                if s in prev_callables:
#                         callables.remove(s)
                    if s in callables:
                        callables.remove(s)
                    docs.append(s)
                    continue
                if call_condition(s):
                    callables.append(s)
                else:
                    docs.append(s)
        else:
            docs.append(doc)
            callables.remove(doc)
        i+=1             

    return [check_doc(doc) for doc in set(docs) if len(doc)>0]
        

In [681]:
def t5_recursive_call(doc):
    print(doc)
    
    docs = []
    callables = [doc]
    prev_callables=[doc]
    i=0
    
    while len(callables) > 0:
        print(i)
        print("[callables]=", callables)
        print("[docs]", docs)
        if i > 10:
            break
        next_call = []
        for doc in callables:
            make_call = call_condition(doc)
            print("doc: ", doc)
            if make_call:
                splits = t5_split(doc)
                splits = list(set(splits))
#                 callables.remove(doc)
                
                for split in splits:
                    print("split: ", split)
#                     split = check_doc(split)
                    if call_condition(split):
                        next_call.append(split)
                    else:
                        docs.append(split)
            else:
                docs.append(doc)
#                 callables.remove(doc)
        callables = list(set(next_call))
        i+=1             

    return [check_doc(doc) for doc in set(docs) if len(doc)>0]
        

In [508]:
# !pip install textacy

In [421]:
# complex_sentence = 'In this image there are two couches with pillows on it, in between them there are tables with some objects on it, there is a fire place on which there are two frames hanging on the wall, beside that there is a lamp, beside that there is a flower pot and other object on the table, there is another frame hanging on the wall, beside that there is a glass window through which we can see trees, grass and plants'

In [422]:
print(complex_sentence)
t5_recursive_call(complex_sentence)

In this image there are a few food items are arranged in the rack, behind it there is a person standing and holding some object in his hand, there are a few objects on the table, beside that there is like a refrigerator and there are a few bottles and other objects are arranged in the rack
0
[callables]= ['In this image there are a few food items are arranged in the rack, behind it there is a person standing and holding some object in his hand, there are a few objects on the table, beside that there is like a refrigerator and there are a few bottles and other objects are arranged in the rack']
[docs] []


1
[callables]= ['In this image there are a few food items are arranged in the rack behind it there is a person standing and holding some object in his hand', 'There are a few bottles and other objects are arranged in the rack']
[docs] ['There are a few objects on the table beside that there is like a refrigerator']


2
[callables]= ['There are a few bottles and other objects are arran

['There are a few bottles',
 'There are a few objects on the table beside that there is like a refrigerator',
 'he holds some object in his hand',
 'Behind it there is a person standing and holding some object in his hand',
 'In this image there are a few food items are arranged in the rack',
 'Other objects are arranged in the rack']

In [214]:
d = 'There is a depiction of a duck and there is a depiction of a duck'
d.split("and")

['There is a depiction of a duck ', ' there is a depiction of a duck']

In [ ]:
%%time
t5_split(complex_sentence)

In [247]:
%%time
# complex_sentence = "This comedy drama is produced by Tidy , the company she co-founded in 2008 with her husband David Peet , who is managing director ."
# complex_sentence = "In this image there are chairs, sofa, in the background there is a desk, pillar and a wall, for that wall there are curtains and there is an object, at the top there is a ceiling and lights"
# complex_sentence = "In the image i can see a stairs,wooden drawer,shelves,bottles,sofa set,switches,photo frame and some other objects."


sample_tokenized = tokenizer(complex_sentence, return_tensors="pt").to(device)
answer = model.generate(sample_tokenized['input_ids'], attention_mask = sample_tokenized['attention_mask'], max_length=256, num_beams=5)
gene_sentence = tokenizer.decode(answer[0], skip_special_tokens=True)
complex_sentence, gene_sentence

CPU times: user 2.86 s, sys: 16 ms, total: 2.88 s
Wall time: 2.88 s


('In this image there are sofas with pillows on it, in front of that there is a table with some plant pot on it, beside the sofa there is a lamp and in the background there is a fireplace, beside that there is a tree and there is a depiction of a duck, there is a frame hanging on the wall and on the left side of the image we can see the leaves of a tree',
 'In this image there are sofas with pillows on it, in front of that there is a table with some plant pot on it and in the background there is a fireplace, beside that there is a tree and there is a depiction of a duck.  On the left side of the image we can see the leaves of a tree and on the left side of the image we can see the leaves of a tree.')

In [360]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/roberta2roberta_L-24_wikisplit")
model = AutoModelForSeq2SeqLM.from_pretrained("google/roberta2roberta_L-24_wikisplit")

In [366]:
complex_sentence
input_ids = tokenizer(tokenizer.bos_token + complex_sentence + tokenizer.eos_token, return_tensors="pt").input_ids
output_ids = model.generate(input_ids)[0]
print(tokenizer.decode(output_ids, skip_special_tokens=True))
# should output

On the right side of the image there is a table, there is a box, there is a box, there is a toy and there are posters on the wall, there are numbers and symbols on the posters and there are a picture of a tree, there are pictures of a tree, there are pictures of people on the wall, there are windows and curtains. There are windows and curtains and there is a tree behind the window.


In [367]:
complex_sentence

'On the right side of the image there is a table and there is a box and there is a toy and there are posters on the wall and there are numbers and symbols on the posters and there is a picture of a tree and there are pictures of fruits and there are pictures of people on the wall and there are windows and curtains and there is a tree behind the window'

In [997]:
# doc = nlp(complex_sentence)
# pos_answers = []

# print(doc,"\n")
# for word in doc:
# #     if word.tag_ in ['NN','JJ','NNS']:
#         pos_answers.append(word.text)
#         print(word.text, word.tag_, word.dep_, ", Explanation:", spacy.explain(word.tag_))

In [40]:
i = 0

def generate_splits(row):
    global i
    caption = row['caption']
    tokenized_caption = tokenizer(caption, return_tensors="pt").to(device)
    answer = model.generate(tokenized_caption['input_ids'],
                            attention_mask = tokenized_caption['attention_mask'], 
                            max_length=256, num_beams=5)
    answer = tokenizer.decode(answer[0], skip_special_tokens=True)
    answer = nlp(answer)
    splits = [sent.text.strip() for sent in answer.sents]
    row['split_caption'] = splits
    if not i%100: print(i)
    i+=1
    return row

train_df = train_df.apply(generate_splits, axis=1)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [57]:
list(train_df.split_caption)[5]

['In this image I can see washroom and I can also see floor, bathtub, wash basin taps, window glass shower hose and some object.',
 'The background is dark and the background is dark.']

In [59]:
list(train_df.caption)[5]

'In this image I can see washroom and I can also see floor, bathtub, wash basin, taps, window, glass, shower, hose, some object, wall and the background is dark.'

In [70]:
i=0
val_df = val_df.apply(generate_splits, axis=1)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000


In [75]:
from pprint import pprint

In [122]:
split_samples = list(train_df.split_caption)
# pprint(split_samples)

In [975]:
# (list(train_df.split_caption)[-1000:-1])

In [114]:
# !cat ../data/ADE20K-pairs/imgs/training/cultural/apse__indoor/ADE_train_00001473.json

In [117]:
t_c = train_df.caption
len(t_c)

20476